In [1]:
import pandas as pd

In [2]:
df = pd.read_json(path_or_buf='gg2013.json')
# df = pd.read_json(path_or_buf='gg2015.json')
df.head(3)

,id,text,timestamp_ms,user
0,290620657987887104,JLo's dress! #eredcarpet #GoldenGlobes,2013-01-14 00:45:38,"{'screen_name': 'Dozaaa_xo', 'id': 557374298}"
1,290620657887219713,What's making Sofia Vergara's boobs stay like ...,2013-01-14 00:45:38,"{'screen_name': 'theAmberShow', 'id': 14648726}"
2,290620657828524032,RT @FabSugar: Kerry Washington is EVERYTHING. ...,2013-01-14 00:45:38,"{'screen_name': 'SweetyPW', 'id': 35498686}"


In [3]:
data = df['text']
data[:5]

0               JLo's dress! #eredcarpet #GoldenGlobes
1    What's making Sofia Vergara's boobs stay like ...
2    RT @FabSugar: Kerry Washington is EVERYTHING. ...
3       Anne Hathaway has got me living. #GoldenGlobes
4    Jennifer Lopez's lace dress? Thoughts? #Golden...
Name: text, dtype: object

In [4]:
len(data)

174643

In [5]:
# sample data if necessary
sample_size = 200000
if len(df) > sample_size:
    data = data.sample(n=sample_size)
len(data)

174643

In [6]:
# hard-coded awards
awards = ['Best Motion Picture - Drama',
         'Best Motion Picture - Musical or Comedy',
         'Best Performance by an Actress in a Motion Picture - Drama',
         'Best Performance by an Actor in a Motion Picture - Drama',
         'Best Performance by an Actress in a Motion Picture - Musical or Comedy',
         'Best Performance by an Actor in a Motion Picture - Musical or Comedy',
         'Best Performance by an Actress in a Supporting Role in any Motion Picture',
         'Best Performance by an Actor in a Supporting Role in any Motion Picture',
         'Best Director - Motion Picture',
         'Best Screenplay - Motion Picture',
         'Best Motion Picture - Animated',
         'Best Motion Picture - Foreign Language',
         'Best Original Score - Motion Picture',
         'Best Original Song - Motion Picture',
         'Best Television Series - Drama',
         'Best Television Series - Musical or Comedy',
         'Best Television Limited Series or Motion Picture Made for Television',
         'Best Performance by an Actress in a Limited Series or a Motion Picture Made for Television',
         'Best Performance by an Actor in a Limited Series or a Motion Picture Made for Television',
         'Best Performance by an Actress In A Television Series - Drama',
         'Best Performance by an Actor In A Television Series - Drama',
         'Best Performance by an Actress in a Television Series - Musical or Comedy',
         'Best Performance by an Actor in a Television Series - Musical or Comedy',
         'Best Performance by an Actress in a Supporting Role in a Series, Limited Series or Motion Picture Made for Television',
         'Best Performance by an Actor in a Supporting Role in a Series, Limited Series or Motion Picture Made for Television',
         'Cecil B. deMille Award']

In [7]:
import re

In [8]:
def reduce(line):
    # remove words "best", "performance", "motion", "picture", "limited", "series", "language", "role", in", "a", "an", "any", "made", "for", "by", "b.", "award", and all punctuations
    pattern = r'\bbest\b|\bperformance\b|\bmotion\b|\bpicture\b|\blimited\b|\bseries\b|\blanguage\b|\brole\b|\bin\b|\ba\b|\ban\b|\bany\b|\bmade\b|\bfor\b|\bby\b|\bb\b|\baward\b|[^\w\s]'
    return re.sub(pattern, ' ', line.lower()).split()

awards_reduced = [sorted(set(reduce(award)), key=lambda word: reduce(award).index(word)) for award in awards]

# test
# for award in awards:
#     ls = reduce(award)
#     print(sorted(set(ls), key=lambda word: ls.index(word)))

In [9]:
def generate_award_num_keywords_map(awards):
    awards_reduced = []
    award_num_keywords_map = {}
    for i, award in enumerate([sorted(set(reduce(award)), key=lambda word: reduce(award).index(word)) for award in awards]):
        if 'or' in award:
            # find number of words from "or" to the end of award name
            num_words = len(award) - 1 - award.index('or')
            award.remove('or')
            # do not count words from "or" to the end of award name
            award_num_keywords_map[i] = len(award) - num_words 
        else:
            award_num_keywords_map[i] = len(award)
        awards_reduced.append(award)
    return award_num_keywords_map, awards_reduced

In [30]:
award_num_keywords_map, awards_reduced = generate_award_num_keywords_map(awards)

pairs = sorted(award_num_keywords_map.items(), key=lambda pair: pair[0])

In [31]:
for pair in pairs:
    print(awards_reduced[pair[0]])
    print(pair)
    print

['drama']
(0, 1)
['musical', 'comedy']
(1, 1)
['actress', 'drama']
(2, 2)
['actor', 'drama']
(3, 2)
['actress', 'musical', 'comedy']
(4, 2)
['actor', 'musical', 'comedy']
(5, 2)
['actress', 'supporting']
(6, 2)
['actor', 'supporting']
(7, 2)
['director']
(8, 1)
['screenplay']
(9, 1)
['animated']
(10, 1)
['foreign']
(11, 1)
['original', 'score']
(12, 2)
['original', 'song']
(13, 2)
['television', 'drama']
(14, 2)
['television', 'musical', 'comedy']
(15, 2)
['television']
(16, 1)
['actress', 'television']
(17, 1)
['actor', 'television']
(18, 1)
['actress', 'television', 'drama']
(19, 3)
['actor', 'television', 'drama']
(20, 3)
['actress', 'television', 'musical', 'comedy']
(21, 3)
['actor', 'television', 'musical', 'comedy']
(22, 3)
['actress', 'supporting', 'television']
(23, 2)
['actor', 'supporting', 'television']
(24, 2)
['cecil', 'demille']
(25, 2)


In [12]:
# test
'actress' not in awards_reduced[2]

False

In [13]:
retweets_freq_dict = {}

def remove_retweet_prefix(line):
    # find 'RT @abc: ' where abc's length is arbitrary
    pattern = re.compile(r'\bRT @([\w\'/]*)\b: ') 
    match = re.search(pattern, line)
    if match:
        # store corresponding retweet without 'RT @' prefix
        string = match.group()[4:]
        if string in retweets_freq_dict:
            retweets_freq_dict[string] += 1
        else:
            retweets_freq_dict[string] = 1
    return re.sub(pattern, ' ', line)

In [14]:
hashtag_freq_dict = {}

def remove_hashtag(line):
    pattern = re.compile(r'#([\w\'/]*)\b')
    matches = re.findall(pattern, line)
    if matches:
        # store corresponding hashtag
        for match in matches:
            if match in hashtag_freq_dict:
                hashtag_freq_dict[match] += 1
            else:
                hashtag_freq_dict[match] = 1
    return re.sub(pattern, ' ', line)

In [15]:
at_freq_dict = {}

def remove_at(line):
    pattern = re.compile(r'@([\w\'/]*)\b')
    matches = re.findall(pattern, line)
    if matches:
        # store corresponding hashtag
        for match in matches:
            if match in at_freq_dict:
                at_freq_dict[match] += 1
            else:
                at_freq_dict[match] = 1
    return re.sub(pattern, ' ', line)

In [16]:
def remove_url(line):
    pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+\b')
    return re.sub(pattern, ' ', line)

In [17]:
def cleanse(line):
    # replace everything to ' ' except alphanumeric character, whitespace, apostrophe, hashtag
    return re.sub(r'[^\w\s\'#]', ' ', line)

In [18]:
cleansed_data = []
for tweet in data:
    line = remove_retweet_prefix(tweet)
    line = remove_hashtag(line)
    line = remove_at(line)
    line = remove_url(line)
    line = cleanse(line)
    cleansed_data.append(line)

In [19]:
import spacy
# python3 -m spacy download en
nlp = spacy.load('en')

In [20]:
def identify_entities(text):
    entities = list(nlp(text).ents)
    tags = {}
    for entity in entities:
        if entity not in tags:
            tags[entity.text]=[entity.label_]
    return tags

In [21]:
# some ideas to improve spaCy's Name Entity Recognizer
# https://github.com/explosion/spaCy/issues/1655
# https://github.com/explosion/spacy/blob/master/examples/training/train_new_entity_type.py
# https://spacy.io/usage/training#section-ner

In [32]:
# %%time
# # test
# num = 0
# pattern = re.compile(r'(\bwin)|(\bwon\b)|(\bbest\b)')
# pattern2 = re.compile(r'\bdrama\b')
# # pattern3 = re.compile(r'\bmovie\b')
# e = {}
# for line in cleansed_data:    
#     match = re.findall(pattern, line.lower())    
#     match2 = re.findall(pattern2, line.lower())  
#     match3 = re.findall(pattern3, line.lower())  
#     if match and match2:
#         tags = identify_entities(line)
#         # check the first 10 cases
#         if count < 10:
#             print(num)
#             print(match, match2)
#             print(line)
#             print(tags.keys())
#             print()
        
#         for entity in tags.keys():
#             entity = entity.strip() 
#             if len(entity) > 1:    
#                 if entity not in e:
#                     e[entity] = 1
#                 else:
#                     e[entity] += 1
#         num += 1
    
# print(count)
# print(sorted(e.items(), key=lambda pair: pair[1], reverse=True)[:100])

In [72]:
def find_award_winner(awards_reduced, award_index, verbose=False):
    print('Predicting for:', awards[award_index])    
    award = awards_reduced[award_index]
    print('award name reduced:', award)
    num_keywords_to_match = award_num_keywords_map[award_index]
    print("num keywords to match:", num_keywords_to_match)
    
    # add word boundary '\b' to prevent grabbing examples like "showing" and "wonder"    
    pattern = re.compile(r'(\bwin)|(\bwon\b)|(\bbest\b)', re.IGNORECASE)
        
    entity_dict = {}
    
    num = 0
    flag = 0
    # if len(award) == num_keywords_to_match and len(award) != 2, the first word in award must be included
    if len(award) != num_keywords_to_match and len(award) != 2:
        flag = 1
        print("matching first word '{0}'".format(award[0]))
        first_word = re.compile(r'\b{0}\b'.format(award[0]), re.IGNORECASE)
       
        for line in cleansed_data:
            match = re.findall(pattern, line.lower())
            match_first_word = re.findall(first_word, line.lower())
            # if line contains at least number keywords to match and pattern is found
            if len(set(award[1:]).intersection(set(line.split()))) == num_keywords_to_match and match and match_first_word:
                tags = identify_entities(line)

                if verbose:
                    # print the first 5 occurrences
                    if num < 5:
                        print(match)
                        print(line)
                        print(tags.keys())
                        print()

                for entity in tags.keys():
                    entity = entity.strip() 
                    if len(entity) > 1:    
                        if entity not in entity_dict:
                            entity_dict[entity] = 1
                        else:
                            entity_dict[entity] += 1
                num += 1
    
    else:
        for line in cleansed_data:
            match = re.findall(pattern, line.lower())
            # if line contains at least number keywords to match and pattern is found
            if len(set(award).intersection(set(line.split()))) == num_keywords_to_match and match:
                tags = identify_entities(line)

                if verbose:
                    # print the first 5 occurrences
                    if num < 5:
                        print(match)
                        print(line)
                        print(tags.keys())
                        print()

                for entity in tags.keys():
                    entity = entity.strip() 
                    if len(entity) > 1:    
                        if entity not in entity_dict:
                            entity_dict[entity] = 1
                        else:
                            entity_dict[entity] += 1
                num += 1
    
    # if no results found, recursively reducing num_keywords_to_match
    while(num == 0):
        print('no results found!!! reducing num_keywords_to_match!!!')
        num_keywords_to_match -= 1
        if num_keywords_to_match == 0:
            break
            
        if flag == 1:
            print("matching first word '{0}'".format(award[0]))
            for line in cleansed_data:
                match = re.findall(pattern, line.lower())
                match_first_word = re.findall(first_word, line.lower())
                # if line contains at least number keywords to match and pattern is found
                if len(set(award[1:]).intersection(set(line.split()))) == num_keywords_to_match and match and match_first_word:
                    tags = identify_entities(line)

                    if verbose:
                        # print the first 5 occurrences
                        if num < 5:
                            print(match)
                            print(line)
                            print(tags.keys())
                            print()

                    for entity in tags.keys():
                        entity = entity.strip() 
                        if len(entity) > 1:    
                            if entity not in entity_dict:
                                entity_dict[entity] = 1
                            else:
                                entity_dict[entity] += 1
                    num += 1

        else:
            for line in cleansed_data:
                match = re.findall(pattern, line.lower())
                # if line contains at least number keywords to match and pattern is found
                if len(set(award).intersection(set(line.split()))) == num_keywords_to_match and match:
                    tags = identify_entities(line)

                    if verbose:
                        # print the first 5 occurrences
                        if num < 5:
                            print(match)
                            print(line)
                            print(tags.keys())
                            print()

                    for entity in tags.keys():
                        entity = entity.strip() 
                        if len(entity) > 1:    
                            if entity not in entity_dict:
                                entity_dict[entity] = 1
                            else:
                                entity_dict[entity] += 1
                    num += 1
        
        
    print('num of matches:', num)
    return entity_dict

In [62]:
%%time
# test
entity_dict = find_award_winner(awards_reduced, 0, verbose=True)

Predicting for: Best Motion Picture - Drama
award name reduced: ['drama']
num keywords to match: 1
[('', '', 'best')]
Give me Lincoln for Best Movie Drama  Silver Linings Playbooks for Comedy Musical  Homeland for Tv drama and Girls for comedy   
dict_keys(['Lincoln for Best Movie Drama  Silver Linings Playbooks for Comedy Musical  Homeland for Tv'])

[('', '', 'best')]
  and the   for best supporting actor in a drama goes to  Christof Waltz 
dict_keys(['  ', ' Christof Waltz'])

[('win', '', ''), ('', '', 'best')]
Maggie Smith wins for best supporting actress in a drama  Go Professor McGonnagall     
dict_keys(['Maggie Smith', ' Go', 'McGonnagall'])

[('', '', 'best')]
Congrats to the Dowager Countess      Best Supporting Actress in a drama series  
dict_keys([])

[('win', '', ''), ('', 'won', ''), ('', '', 'best')]
    Is Leo Dicaprio ever going to win     he won for best actor drama 4 the departed
dict_keys(['Leo Dicaprio', '4'])

num of matches: 810
CPU times: user 34.3 s, sys: 4.0

In [25]:
from fuzzywuzzy import fuzz

In [56]:
def remove_goldeb_globes(top_results, entity_freq_dict):
    golden_globes = [name for name in [pair[0] for pair in top_results] if fuzz.ratio(name.lower(), 'golden globes') > 60]
    for name in golden_globes:
        if name in entity_freq_dict:
            del entity_freq_dict[name]
    return entity_freq_dict


# we also consider dropping all lower cases examples or examples that contain digit(s), which are not names
def filter_names(pair_list, entity_freq_dict):
    filtered_results = []
    for pair in pair_list:
        string = ''.join(pair[0].split())
        if not all(char.islower() for char in string) and not any(char.isdigit() for char in string):
            filtered_results.append(pair)
        else:
            if pair[0] in entity_freq_dict:
                del entity_freq_dict[pair[0]]
    return filtered_results, entity_freq_dict


def merge_names(top_results, entity_freq_dict):
    names = [pair[0] for pair in top_results]
    names_clusters = []
    for name in names:
        # each name starts as a cluster
        cluster = [name]
        names_to_reduce = names[:]
        names_to_reduce.remove(name)
        # one vs. all comparisons
        for i in names_to_reduce:
            ratio = fuzz.ratio(name.lower(), i.lower())
            # if similarity is larger than 75 or one name is contained in the other name
            if ratio > 75 or re.search(name, i, flags=re.IGNORECASE) or re.search(i, name, flags=re.IGNORECASE):
                cluster.append(i)
        # if multiple names are identified in one cluster
        if len(cluster) > 1:
            names_clusters.append(cluster)

    # find names clusters that should merge
    # ['Amy Poehler', 'Amy', 'Amy Poelher']
    # ['Tina', 'Tina Fey']

    # sort clusters
    names_clusters.sort()
    # sort within each cluster
    names_clusters = ['|'.join(sorted(cluster)) for cluster in names_clusters]
    # remove overlaps
    names_clusters_reduced = [line.split('|') for line in list(set(names_clusters))]
    # sort by length from shortest to longest (merge from the shortest)
    names_clusters_reduced.sort(key=len)

    # weighted frequency of an entity is defined by its frequency multiplied by its string length
    def weighted_freq(element):
        return entity_freq_dict[element] * len(element)

    e = entity_freq_dict.copy()
    for cluster in names_clusters_reduced:
        # select the entity name with highest weighted frequency
        selected_entity_name = max(cluster, key=weighted_freq)
        cluster.remove(selected_entity_name)
        # for names to be merged to the selected entity name
        for name in cluster:
            # if not deleted in previous cases, cumulate frequencies to the selected entity
            if name in e and selected_entity_name in e:
                e[selected_entity_name] += e[name]
                del e[name]

    top_10 = sorted(e.items(), key=lambda pair: pair[1], reverse=True)[:10]
    return top_10

# Predictions for Award Winner

In [73]:
%%time
for key in award_num_keywords_map.keys():
    entity_freq_dict = find_award_winner(awards_reduced, key, verbose=False)
    top_results = sorted(entity_freq_dict.items(), key=lambda pair: pair[1], reverse=True)[:100]
    # remove 'golden globes' from identified host names
    entity_freq_dict = remove_goldeb_globes(top_results, entity_freq_dict)
    top_results = sorted(entity_freq_dict.items(), key=lambda pair: pair[1], reverse=True)[:30]
    # filter for names if necessary
    if 'actor' in awards_reduced[key] or 'actress' in awards_reduced[key] or 'director' in awards_reduced[key]:
        top_results, entity_freq_dict = filter_names(top_results, entity_freq_dict)
    print('top results:')
    print(top_results)
    print('top results after merging:')
    top_10 = merge_names(top_results, entity_freq_dict)
    print(top_10)
    print()

Predicting for: Best Motion Picture - Drama
award name reduced: ['drama']
num keywords to match: 1
num of matches: 810
top results:
[('Homeland', 164), ('Jessica Chastain', 79), ('Daniel Day Lewis', 62), ('Lincoln', 61), ('Claire Danes', 59), ('Zero', 44), ('Dark Thirty', 39), ('Argo', 39), ('first', 34), ('Damien Lewis', 30), ("Les Miz'", 29), ('Lewis  Chastain', 29), ('Damian Lewis', 25), ('night', 24), ('15', 22), ('Ben Affleck', 20), ('for', 13), ('Zero Dark Thirty', 12), ('Kathryn Bigelow', 10), ('Latest', 10), ('Breaking', 9), ('12 year old', 9), ('Best', 8), ('Damian', 8), ('A lanky', 8), ('Lewis', 7), ('Breaking Bad', 7), ('Best TV', 6), ('WINNER', 6), ('RT', 6)]
top results after merging:
[('Homeland', 164), ('Daniel Day Lewis', 161), ('Dark Thirty', 101), ('Jessica Chastain', 79), ('Lincoln', 61), ('Claire Danes', 59), ('Argo', 39), ('first', 34), ("Les Miz'", 29), ('night', 24)]

Predicting for: Best Motion Picture - Musical or Comedy
award name reduced: ['musical', 'comedy'

num of matches: 137
top results:
[('Quentin Tarantino', 77), ('Django Unchained', 31), ('Django', 12), ('Tarantino', 10), ('WINNER', 4), ('Quentin', 4), ('Except', 4), ('QT', 3), ('tonight', 3), ('Lincoln', 2), ('DJANGO', 2), ('Mhm  Tarantino', 1), ('quentin', 1), ('Herman Munster', 1), ('Very', 1), ('2nd', 1), ('Surprised', 1), ('Wanna', 1), ('one day', 1), ('XD', 1), ('Basterds', 1), ('one', 1), ('Suck', 1), ('Hell', 1), ('3028', 1), ('the night', 1), ('a couple of years ago', 1), ('Quentin Tarantino DJANGO', 1), ("Django's", 1), ('Will Lincoln', 1)]
top results after merging:
[('Quentin Tarantino', 94), ('Django Unchained', 46), ('WINNER', 4), ('Except', 4), ('QT', 3), ('tonight', 3), ('Lincoln', 3), ('one day', 2), ('Herman Munster', 1), ('Very', 1)]

Predicting for: Best Motion Picture - Animated
award name reduced: ['animated']
num keywords to match: 1
num of matches: 147
top results:
[('Brave', 43), ('Ralph', 9), ('Best', 7), ('Wreck', 7), ('Congratulations to Brave', 5), ('Agre

no results found!!! reducing num_keywords_to_match!!!
matching first word 'actress'
num of matches: 124
top results:
[('Jennifer Lawrence', 53), ('Lena Dunham', 52), ('Girls', 29), ('Lawrence', 7), ('Best Actress', 7), ('WINNER', 5), ('Meryl', 4), ('Meryl Streep', 3), ('Silver Linin', 3), ('JLAW', 2), ('RT', 2), ('Streep', 1), ('Jennifer', 1), ('Lawrence Silver', 1), ('Hmm', 1), ('Jessica Chastain', 1), ('Oscar', 1), ('Heard', 1), ('Silver Lingings Playbook', 1), ('Yay', 1), ('Silver Linings Playbook', 1), ('Silver Lining Playbook', 1), ('Congratulations', 1), ('Kentucky', 1), ('GIRLS', 1), ('HBO', 1), ('for Girls', 1), ('Congrats', 1)]
top results after merging:
[('Jennifer Lawrence', 62), ('Lena Dunham', 52), ('Girls', 31), ('Meryl Streep', 8), ('Best Actress', 7), ('Silver Linin', 6), ('WINNER', 5), ('JLAW', 2), ('RT', 2), ('Hmm', 1)]

Predicting for: Best Performance by an Actor in a Television Series - Musical or Comedy
award name reduced: ['actor', 'television', 'musical', 'comed